# Lab 09: Image Generation with DALL-E

## Overview
In this lab, you'll learn how to use Azure OpenAI's DALL-E model to generate images from text descriptions. DALL-E is a powerful AI system that can create realistic images and art from natural language prompts.

## Learning Objectives
- Connect to Azure OpenAI DALL-E service
- Generate images from text prompts
- Save and display generated images
- Understand prompt engineering for image generation
- Explore different styles and artistic effects

## Prerequisites
- Azure OpenAI resource with DALL-E deployment
- API credentials configured in `.env` file

## Step 1: Setup and Configuration

In [ ]:
# Install required packages
!pip install openai python-dotenv pillow requests -q

In [ ]:
import os
import json
import requests
from datetime import datetime
from pathlib import Path
from dotenv import load_dotenv
from openai import AzureOpenAI
from PIL import Image
from IPython.display import display, HTML

print("✓ Packages imported successfully")

## Step 2: Load Configuration

Load the Azure OpenAI credentials from the `.env` file.

In [ ]:
# Load environment variables from the python subfolder
load_dotenv('python/.env')

endpoint = os.getenv("ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT")
api_version = os.getenv("API_VERSION")

if not endpoint or not model_deployment:
    print("⚠️  Please configure ENDPOINT and MODEL_DEPLOYMENT in python/.env file")
else:
    print(f"✓ Configuration loaded")
    print(f"  Endpoint: {endpoint[:50]}...")
    print(f"  Model Deployment: {model_deployment}")
    print(f"  API Version: {api_version}")

## Step 3: Initialize DALL-E Client

Create a connection to the Azure OpenAI DALL-E service.

In [ ]:
# Initialize Azure OpenAI client
# Note: You'll need to set AZURE_OPENAI_API_KEY in environment or use DefaultAzureCredential
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=os.getenv("AZURE_OPENAI_API_KEY")  # Make sure this is set in your environment
)

print("✓ Azure OpenAI DALL-E client initialized successfully")

## Step 4: Helper Functions

Create utility functions to generate, save, and display images.

In [ ]:
def generate_image(prompt, size="1024x1024", quality="standard", n=1):
    """
    Generate an image using DALL-E.
    
    Args:
        prompt: Text description of the image to generate
        size: Image size ("1024x1024", "1792x1024", or "1024x1792")
        quality: "standard" or "hd"
        n: Number of images to generate (1-10)
    
    Returns:
        List of image URLs
    """
    print(f"🎨 Generating image...")
    print(f"   Prompt: {prompt}")
    print(f"   Size: {size}, Quality: {quality}\n")
    
    result = client.images.generate(
        model=model_deployment,
        prompt=prompt,
        size=size,
        quality=quality,
        n=n
    )
    
    image_urls = [image.url for image in result.data]
    return image_urls

def save_image(image_url, filename=None):
    """
    Download and save an image from a URL.
    
    Args:
        image_url: URL of the image to download
        filename: Optional custom filename
    
    Returns:
        Path to saved image
    """
    # Create images directory if it doesn't exist
    image_dir = Path("images")
    image_dir.mkdir(exist_ok=True)
    
    # Generate filename if not provided
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"dalle_image_{timestamp}.png"
    
    # Ensure .png extension
    if not filename.endswith('.png'):
        filename += '.png'
    
    image_path = image_dir / filename
    
    # Download and save
    response = requests.get(image_url)
    with open(image_path, "wb") as f:
        f.write(response.content)
    
    print(f"✓ Image saved: {image_path}")
    return image_path

def display_image_from_url(image_url, width=512):
    """Display an image from a URL in the notebook."""
    response = requests.get(image_url)
    img = Image.open(requests.get(image_url, stream=True).raw)
    
    # Resize for display
    aspect_ratio = img.height / img.width
    new_height = int(width * aspect_ratio)
    img_resized = img.resize((width, new_height))
    
    display(img_resized)

def generate_and_display(prompt, size="1024x1024", quality="standard", save=True):
    """Generate, display, and optionally save an image."""
    image_urls = generate_image(prompt, size, quality)
    
    for i, url in enumerate(image_urls, 1):
        print(f"\n📷 Image {i}:")
        display_image_from_url(url)
        
        if save:
            save_image(url)
    
    return image_urls

print("✓ Helper functions defined")

## Step 5: Generate Your First Image

Let's create a simple image with a basic prompt.

In [ ]:
# Basic prompt
prompt = "A photorealistic image of fresh tropical fruits arranged on a wooden table"

image_urls = generate_and_display(prompt)

## Step 6: Explore Different Styles

DALL-E can generate images in various artistic styles. Let's experiment!

In [ ]:
# Watercolor style
prompt = "A watercolor painting of a basket of fresh mangoes and oranges"
print("🎨 Style: Watercolor Painting\n")
generate_and_display(prompt)

In [ ]:
# Oil painting style
prompt = "An oil painting in the style of Van Gogh, depicting a fruit market with vibrant colors"
print("🎨 Style: Oil Painting (Van Gogh)\n")
generate_and_display(prompt)

In [ ]:
# Digital art style
prompt = "A futuristic digital art illustration of a smart grocery store with holographic fruit displays"
print("🎨 Style: Digital Art\n")
generate_and_display(prompt)

In [ ]:
# Minimalist style
prompt = "A minimalist flat design illustration of a single mango, clean and simple, pastel colors"
print("🎨 Style: Minimalist\n")
generate_and_display(prompt)

## Step 7: Detailed Prompts for Better Results

More detailed prompts generally produce better results. Include:
- Subject matter
- Style or medium
- Lighting and atmosphere
- Colors and mood
- Composition details

In [ ]:
# Detailed prompt example
prompt = """A professional product photography shot of exotic tropical fruits including dragon fruit, 
passion fruit, and star fruit, arranged artistically on a marble surface. Soft natural lighting from 
the side, shallow depth of field, vibrant colors, shot with a macro lens, food photography style, 
high resolution, studio quality"""

print("📸 Detailed Product Photography\n")
generate_and_display(prompt, quality="hd")

## Step 8: Create Images for Different Use Cases

Let's generate images suitable for various applications.

In [ ]:
# Marketing banner
prompt = """A wide banner image for a healthy eating campaign, featuring fresh organic fruits 
and vegetables, bright and cheerful atmosphere, text space in the center, 16:9 aspect ratio, 
professional marketing design"""

print("📱 Marketing Banner\n")
generate_and_display(prompt, size="1792x1024")

In [ ]:
# Social media post
prompt = """An Instagram-worthy flat lay photograph of a healthy breakfast smoothie bowl 
topped with fresh mango, berries, and granola, surrounded by tropical fruits, 
bright natural lighting, square format, food blogger aesthetic"""

print("📸 Social Media Post\n")
generate_and_display(prompt)

In [ ]:
# Educational illustration
prompt = """An educational diagram showing the cross-section of a mango fruit, 
labeled with parts like seed, flesh, and skin, scientific illustration style, 
clean and clear, suitable for a textbook"""

print("📚 Educational Illustration\n")
generate_and_display(prompt)

## Step 9: Creative and Artistic Prompts

Get creative with imaginative and artistic concepts!

In [ ]:
# Surreal art
prompt = """A surreal digital art piece: a giant mango floating in space with smaller fruits 
orbiting around it like planets, cosmic background with stars and nebulae, 
vibrant colors, dreamlike atmosphere"""

print("🌌 Surreal Art\n")
generate_and_display(prompt)

In [ ]:
# Fantasy theme
prompt = """A whimsical fantasy illustration of a magical fruit garden where fruits glow with 
inner light, fairy lights floating around, enchanted atmosphere, storybook art style, 
warm and inviting colors"""

print("✨ Fantasy Theme\n")
generate_and_display(prompt)

## Step 10: Interactive Image Generation

Create your own images with custom prompts!

In [ ]:
def interactive_generation():
    """Interactive DALL-E image generation session."""
    print("="*80)
    print("🎨 Interactive DALL-E Image Generation")
    print("="*80)
    print("\n💡 Tips for great prompts:")
    print("  • Be specific about style, colors, and composition")
    print("  • Include lighting and atmosphere details")
    print("  • Mention artistic style or photography type")
    print("  • Use descriptive adjectives")
    print("\nType 'quit' to exit\n")
    print("="*80 + "\n")
    
    image_count = 0
    
    while True:
        prompt = input("\n🎨 Enter your image prompt: ").strip()
        
        if prompt.lower() == 'quit':
            print(f"\n👋 Generated {image_count} images. Thank you!")
            break
        
        if not prompt:
            print("⚠️  Please enter a prompt.")
            continue
        
        # Ask for quality preference
        quality_input = input("   Quality (standard/hd) [standard]: ").strip().lower()
        quality = quality_input if quality_input in ['standard', 'hd'] else 'standard'
        
        try:
            image_urls = generate_and_display(prompt, quality=quality)
            image_count += len(image_urls)
            print(f"\n✓ Total images generated: {image_count}")
        except Exception as e:
            print(f"\n❌ Error: {e}")
            print("Please try a different prompt.")

# Uncomment to start interactive session
# interactive_generation()

## Step 11: Batch Generation

Generate multiple images from a list of prompts.

In [ ]:
def batch_generate(prompts, size="1024x1024", quality="standard"):
    """Generate multiple images from a list of prompts."""
    results = []
    
    for i, prompt in enumerate(prompts, 1):
        print(f"\n{'='*80}")
        print(f"Generating image {i}/{len(prompts)}")
        print('='*80)
        
        try:
            urls = generate_and_display(prompt, size, quality)
            results.append({
                'prompt': prompt,
                'urls': urls,
                'status': 'success'
            })
        except Exception as e:
            print(f"❌ Failed: {e}")
            results.append({
                'prompt': prompt,
                'error': str(e),
                'status': 'failed'
            })
    
    return results

# Example batch generation
batch_prompts = [
    "A simple line drawing of a mango",
    "A pixel art icon of an orange, 32x32 style",
    "A vintage botanical illustration of tropical fruits"
]

print("🔄 Batch Generation:\n")
batch_results = batch_generate(batch_prompts)

# Summary
print("\n" + "="*80)
print("📊 Batch Generation Summary")
print("="*80)
successful = sum(1 for r in batch_results if r['status'] == 'success')
print(f"✓ Successful: {successful}/{len(batch_prompts)}")
print(f"✗ Failed: {len(batch_prompts) - successful}/{len(batch_prompts)}")

## Step 12: View Generated Images

Browse all images you've generated in this session.

In [ ]:
def show_generated_images():
    """Display all generated images from the images directory."""
    image_dir = Path("images")
    
    if not image_dir.exists():
        print("No images directory found.")
        return
    
    image_files = sorted(image_dir.glob("dalle_image_*.png"))
    
    if not image_files:
        print("No generated images found.")
        return
    
    print(f"📁 Found {len(image_files)} generated images:\n")
    
    for img_path in image_files:
        print(f"\n📷 {img_path.name}")
        img = Image.open(img_path)
        display(img.resize((400, int(400 * img.height / img.width))))

show_generated_images()

## Summary

In this lab, you learned how to:

✅ **Connect to Azure OpenAI DALL-E** - Initialize and authenticate with the service  
✅ **Generate images from text** - Create images using natural language descriptions  
✅ **Use different styles** - Explore artistic styles and photography techniques  
✅ **Craft effective prompts** - Write detailed prompts for better results  
✅ **Save and display images** - Work with generated images programmatically  
✅ **Batch processing** - Generate multiple images efficiently  

## Key Takeaways

- **Be specific**: Detailed prompts with style, lighting, and composition details yield better results
- **Include style keywords**: "photorealistic", "oil painting", "watercolor", etc.
- **Describe atmosphere**: Lighting, mood, colors enhance the output
- **Iterate**: Try variations of prompts to refine results
- **Quality matters**: Use "hd" quality for professional applications

## Prompt Engineering Tips

1. Start with the subject
2. Add style or medium
3. Describe lighting and atmosphere
4. Include color palette
5. Specify composition details
6. Mention camera/artistic techniques

## Next Steps

- Explore the **Advanced Notebook** (09-dalle-advanced.ipynb) for sophisticated techniques
- Experiment with different aspect ratios
- Try combining DALL-E with GPT-4 Vision for image editing workflows
- Build applications that generate custom visual content